<a href="https://colab.research.google.com/github/veren4/SMILES_featurization/blob/master/My_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Tutorial](https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pickle

infile1 = open('/content/drive/My Drive/Rostlab internship/7_PyTorch/Tokenized_Dataset', 'rb')
tokenized_dataset = pickle.load(infile1)
infile1.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###First Trial

In [2]:
import torch
import torch.nn as nn

In [3]:
input_dim = 5 #3         # size of the input at each step
hidden_dim = 10 #3       # size of the hidden state and cell state at each step
n_layers = 1          # number of LSTM layers stacked on top of each other

lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

In [4]:
batch_size = 1
seq_len = 1

#inp = torch.randn(batch_size, seq_len, input_dim)
hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)    # tuple

In [5]:
type(inp)

NameError: ignored

In [ ]:
print("Input:", inp.size())
print("Hidden state:", hidden_state.size())
print("Cell state:", cell_state.size())

Now let's see what comes out of the LSTM layer:

In [ ]:
out, hidden = lstm_layer(inp, hidden)
print("Output shape: ", out.shape)
print("Hidden: ", hidden)

##Data preparation (int_tokens)

####Create vocabulary

In [6]:
#len(tokenized_dataset)

In [7]:
alphabet = set()

for i in tokenized_dataset:
  alphabet.update(i)

In [8]:
alphabet    # length: 12

{'(', ')', '1', '2', '3', '4', '=', 'C', 'N', 'O', 'P', 'S'}

####Create indices to vectorize words

In [9]:
dict_token_alphabet = {}
dict_token_alphabet.update({'Padding': 0, 'Unknown': 1})

index = 2
for i in alphabet:
  dict_token_alphabet.update({i: index})
  index = index+1

dict_token_alphabet

{'(': 8,
 ')': 11,
 '1': 5,
 '2': 4,
 '3': 9,
 '4': 2,
 '=': 12,
 'C': 13,
 'N': 7,
 'O': 3,
 'P': 6,
 'Padding': 0,
 'S': 10,
 'Unknown': 1}

####Vectorize SMILES

In [10]:
int_tokens = [None]*len(tokenized_dataset)    # empty list of length 14

for i in range(len(tokenized_dataset)):
  int_tokens[i] = [None]*len(tokenized_dataset[i])

  for j in range(len(tokenized_dataset[i])):
    int_tokens[i][j] = dict_token_alphabet.get(tokenized_dataset[i][j])

[Tutorial](https://github.com/chrisvdweth/ml-toolkit/blob/master/pytorch/notebooks/minimal-example-lstm-input.ipynb)\
Still open: What is a batch for me? For now, I will go with 3 SMILES.\
The sequence length has to be constant within 1 batch. -> Pad accordingly\
Choose Batches with equal length so that I have to do minimal Padding. -> Is there a function for this like in AllenNLP?

In [28]:
batch = [int_tokens[0], int_tokens[1], int_tokens[3]]

pad_to_this = len(max(batch, key=len))    #  length of the longest list in the batch -> All other elements need to be padded to this length

pad_beginning = True

for SMILES in batch:
  while(len(SMILES) < pad_to_this):
    if(pad_beginning == True):
      SMILES.insert(0, 0)
      pad_beginning = !pad_beginning
    else:
      SMILES.append(0)
      pad_beginning = !pad_beginning

In [17]:
import numpy as np

In [31]:
# make a numpy array out of it
batch = np.array(batch)

# make a PyTorch tensor
batch = torch.tensor(batch, dtype=torch.long)

print(batch)
print('The shape of batch is:', batch.shape)

tensor([[13,  8, 13,  8, 13,  8, 13,  3,  6,  8, 12,  3, 11,  8,  3, 11,  3, 11,
          3, 11,  3, 11, 13,  8, 12,  3, 11, 13,  8, 12,  3, 11,  3],
        [ 0, 13, 13, 13,  8, 13, 11, 13,  8, 12,  3, 11, 13,  8, 12,  3, 11,  3,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [13,  5, 12, 13, 13,  8, 12, 13, 13,  8, 12, 13,  5, 11,  3, 11, 13, 13,
         13,  8, 12,  3, 11,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
The shape of batch is: torch.Size([3, 33])


Now we have the first step of having our data in the shape (batch_size, seq_len). to feed it into an LSTM, we still need input_size.

####Embedding layer

In [39]:
torch.manual_seed(1)

Define layer

In [36]:
vocab_size = len(dict_token_alphabet)  # 14
embed_dim = 10 #<- What does the size of the embeddings mean in my case?

word_embedding_layer = nn.Embedding(vocab_size, embed_dim)

Push batch through layer

In [37]:
batch = word_embedding_layer(batch)

print('The shape of batch is:', batch.shape)   # 3, 33, 10
print()
print(batch)

Now we have want we want: (batch_size, seq_len, input_size)

In [44]:
# brauche ich wahrscheinlich nicht:
#float_batch = [[float(token) for token in SMILES] for SMILES in batch]

##Putting my data into a model

####Setup

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [10]:
import platform
print('Using python: ', platform.python_version())
print('Using torch version: ', torch.__version__)
print('Using device: ', device)
# Machine: 2015 13" Macbook Pro, i5 dual core

import torch.nn as nn
import timeit

Using python:  3.6.9
Using torch version:  1.6.0+cu101
Using device:  cpu


####Model

In [52]:
#batch.size()

torch.Size([3, 33, 10])

In [48]:
# choose the input parameters
input_size = batch.shape[2] #33         # max length of SMILES (token sequence)
hidden_dim = 32   # What does this parameter do?
seq_len= batch.shape[0] #2  # What does this parameter do?

# define the model
my_LSTM=nn.LSTM(input_size, hidden_dim)

# initialise the lstm
for p in my_LSTM.parameters():
    nn.init.constant_(p, val=0.3)

In [53]:
# We need to reshape the batch from (batch_size, seq_len, input_size) to (seq_len, batch_size, input_size)
batch = batch.transpose(0,1)

In [56]:
lstm_out, (h_pytorch, _) = my_LSTM(batch)

print('The shape of lstm_out is:', lstm_out.shape) # (seq_len, batch_size, hidden_dim)
print('The shape of h is:', h_pytorch.shape) # (num_layers*num_directions, batch_size, hidden_dim)

The shape of lstm_out is: torch.Size([33, 3, 32])
The shape of h is: torch.Size([1, 3, 32])


In [57]:
import timeit

In [59]:
runs=10**4

print("Time Pytorch LSTM {} runs: {:.3f}s".format(runs, timeit.timeit("my_LSTM(batch)", 
                                       setup="from __main__ import my_LSTM, batch", 
                                       number=runs))
     )

Time Pytorch LSTM 10000 runs: 18.690s
